<a href="https://colab.research.google.com/github/ZoyaAfzal/25-Python-Projects/blob/main/Markov_Chain_Text_Composer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lyricsgenius markovify


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 15.4 MB/s eta 0:00:00
  Created wheel for markovify: filename=markovify-0.9.4-py3-none-any.whl size=18606 sha256=828e2dc96f2dbbf215c312c6ef88be76a3e6ac7cf9fc79248fe4188d489fbdb3
  Stored in directory: /root/.cache/pip/wheels/9c/20/eb/1a3fb93f3132f2f9683e4efd834800f80c53aeddf50e84ae80
Successfully built markovify


In [9]:
import lyricsgenius
import random
import os


In [24]:

import random

class Vertex(object):
    def __init__(self, value):
        self.value = value
        self.adjacent = {}
        self.neighbors = []
        self.neighbors_weights = []

    def __str__(self):
        return self.value + ' '.join([node.value for node in self.adjacent.keys()])

    def add_edge_to(self, vertex, weight=0):
        self.adjacent[vertex] = weight

    def increment_edge(self, vertex):
        self.adjacent[vertex] = self.adjacent.get(vertex, 0) + 1

    def get_adjacent_nodes(self):
        return self.adjacent.keys()

    # initializes probability map
    def get_probability_map(self):
        for (vertex, weight) in self.adjacent.items():
            self.neighbors.append(vertex)
            self.neighbors_weights.append(weight)

    def next_word(self):
        return random.choices(self.neighbors, weights=self.neighbors_weights)[0]



class Graph(object):
    def __init__(self):
        self.vertices = {}

    def get_vertex_values(self):
        return set(self.vertices.keys())

    def add_vertex(self, value):
        self.vertices[value] = Vertex(value)

    def get_vertex(self, value):
        if value not in self.vertices:
            self.add_vertex(value)
        return self.vertices[value]

    def get_next_word(self, current_vertex):
        return self.vertices[current_vertex.value].next_word()

    def generate_probability_mappings(self):
        for vertex in self.vertices.values():
            vertex.get_probability_map()

    def walk_chain(self, num_words):
        current_vertex = random.choice(list(self.vertices.values()))
        output = [current_vertex.value]

        for _ in range(num_words - 1):
            next_vertex = current_vertex.next_word()
            output.append(next_vertex.value)
            current_vertex = next_vertex

        return ' '.join(output)


In [10]:
genius = lyricsgenius.Genius("hjRYdnTFZMpbZ0Gombbc6mob6SFcdSYKEkjvJOQac73HrGnRzLOeNVS-xa3Zipa8")
genius.skip_non_songs = True
genius.excluded_terms = ["(Remix)", "(Live)"]


In [21]:
#Fetching lyrics for a song list

def get_lyrics(song_titles, artist):
    lyrics = ''
    for title in song_titles:
        try:
            song = genius.search_song(title, artist)
            if song:
                lyrics += song.lyrics.replace('\n', ' ') + ' '
        except:
            print(f"Couldn't fetch {title}")
    return lyrics.lower()


In [6]:
# Building Markov Chain from lyrics

def build_markov_model():
    text = ""
    for file in os.listdir('lyrics'):
        with open(f'lyrics/{file}', 'r', encoding='utf-8') as f:
            text += f.read() + '\n'
    return markovify.NewlineText(text)


In [13]:
#Tokenize and buiding the graph

def build_markov_chain(text):
    words = text.split()
    g = Graph()

    prev_word = None
    for word in words:
        current_vertex = g.get_vertex(word)
        if prev_word:
            prev_vertex = g.get_vertex(prev_word)
            prev_vertex.increment_edge(current_vertex)
        prev_word = word

    g.generate_probability_mappings()
    return g



In [14]:
#Generating text using Markov Chain

def compose_text(graph, length=50):
    current_word = random.choice(list(graph.get_vertex_values()))
    output = [current_word]

    for _ in range(length - 1):
        current_vertex = graph.get_vertex(current_word)
        next_vertex = graph.get_next_word(current_vertex)
        current_word = next_vertex.value
        output.append(current_word)

    return ' '.join(output)


In [25]:
songs = [
    'the box',
    'down below',
    'project dreams',
    'die young',
    'boom boom room',
    'high fashion',
    'roll dice',
    'war baby',
    'every season'
]

lyrics_text = get_lyrics(songs, 'Roddy Ricch')
markov_graph = build_markov_chain(lyrics_text)

def prettify(text):
    text = text.capitalize()
    words = text.split()
    output = ''
    for i, word in enumerate(words):
        output += word + ' '
        if (i + 1) % 10 == 0:
            output += '\n'
    return output.strip()

generated_text = markov_graph.walk_chain(num_words=100)
print(prettify(generated_text))



Searching for "the box" by Roddy Ricch...
Done.
Searching for "down below" by Roddy Ricch...
Done.
Searching for "project dreams" by Roddy Ricch...
Done.
Searching for "die young" by Roddy Ricch...
Done.
Searching for "boom boom room" by Roddy Ricch...
Done.
Searching for "high fashion" by Roddy Ricch...
Done.
Searching for "roll dice" by Roddy Ricch...
Done.
Searching for "war baby" by Roddy Ricch...
Done.
Searching for "every season" by Roddy Ricch...
Done.
Slatt, slatt i gotta keep my jewelry wetter than carried 
by one stood ten racks thought you trappin', make it 
on go, yeah) down on the bottom, check your face, 
botox my dawg just caught his neck like a nigga 
gotta keep it look at the meter i'm a mannequin 
i'm a couple p's then i am, i-i-i (ice) so 
annoying i was down below (down below), down below down 
on me, why she don't need her tongue-tied i was 
fightin' fed cases, 'member them birds sending out the mojo 
deals, we throw it one time and the head chef
